In [2]:
import pandas as pd
from pymilvus import connections, db
import numpy as np

from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings import  HuggingFaceEmbeddings
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings)
from langchain_community.vectorstores import milvus
import _csv
from tqdm import tqdm
from langchain_community.vectorstores import Milvus

In [3]:
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

d:\Projects\Milvus\env\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\Projects\Milvus\env\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
connections.connect(host="127.0.0.1", port=19530)

In [7]:
from pymilvus import MilvusClient, DataType

# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://localhost:19530"
)

# 2. Create a collection in quick setup mode
client.create_collection(
    collection_name="quick_setup1",
    dimension=512
)

In [8]:
client.drop_collection(
    collection_name="quick_setup"
)

In [9]:
_csv.field_size_limit(100000000)
chunksize=25000

In [10]:
chunk_check=0
text_splitter = RecursiveCharacterTextSplitter(chunk_size=612, chunk_overlap=100)
id_const=0

In [11]:
for new_csv in pd.read_csv("fixed.csv",chunksize=chunksize,encoding="utf-8"):
    counter = 0
    embed_chunks=[]
    ids_arr = []
    chunk_id_reference = []
    reference = "date of judgement: "+new_csv["date of judgment"]+ " " +"case_title: "+new_csv["case title"]
    new_csv=new_csv['all_text']
    print('created references')
    new_csv.to_csv('temp2.csv')
    print('temp csv created')
    loader = CSVLoader(file_path="temp2.csv", encoding="utf-8", csv_args={'delimiter': ','})
    data = loader.load()
    print('loading done')

created references
temp csv created
loading done


In [12]:
reference

0       date of judgement: 01-Sep-20 case_title: Mukut...
1       date of judgement: 01-Sep-20 case_title: Prash...
2       date of judgement: 01-Sep-20 case_title: Prabh...
3       date of judgement: 01-Sep-20 case_title: Ratis...
4       date of judgement: 01-Sep-20 case_title: Smt.V...
                              ...                        
1030    date of judgement: 16-Sep-20 case_title: Hemra...
1031    date of judgement: 16-Sep-20 case_title: Md. M...
1032    date of judgement: 16-Sep-20 case_title: Arun ...
1033    date of judgement: 16-Sep-20 case_title: Sanja...
1034    date of judgement: 16-Sep-20 case_title: Sohan...
Length: 1035, dtype: object

In [13]:
text_chunks = text_splitter.split_documents(data)

In [14]:
print('TQDM is initialised')
for i in tqdm(range(len(text_chunks))):
    client.upsert(
    collection_name='quick_setup1',
    data=text_chunks[i]
    )

TQDM is initialised


  0%|          | 0/6964 [00:00<?, ?it/s]


TypeError: wrong type of argument 'data',expected 'Dict' or list of 'Dict', got 'Document'

In [15]:
type(text_chunks)

list

In [16]:
text_chunks[0]

Document(page_content=': 0\nall_text: CAV on: 07/07/2020 Pronounced on:01/09/2020 \n 1. Heard learned counsel for the parties through V.C. Since both these writ applications are interconnected; as such, the same are being disposed of by this common order.', metadata={'source': 'temp2.csv', 'row': 0})

In [1]:
import pandas as pd

df = pd.read_csv("judge_database_master_20-23.csv")

C:\Users\asus\AppData\Local\Temp\ipykernel_1868\1724558018.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("judge_database_master_20-23.csv")


In [2]:
print(df.isnull().sum())

document_id               0
case title              201
judges name(s)            0
date of judgment        201
citation            1015354
issues                    0
decision                  0
cited cases               0
all_text               2271
dtype: int64

In [3]:
df_cleaned = df.dropna(subset=['all_text'])

In [4]:
print(df_cleaned.isnull().sum())

document_id               0
case title              163
judges name(s)            0
date of judgment        163
citation            1013084
issues                    0
decision                  0
cited cases               0
all_text                  0
dtype: int64

In [7]:
df[40000:60000].isnull().sum()

document_id             0
case title            104
judges name(s)          0
date of judgment      104
citation            19997
issues                  0
decision                0
cited cases             0
all_text               29
dtype: int64

In [ ]:
df_cleaned.to_csv("judge_database_master_20-23_final.csv", index= False)